In [1]:
import os
import networkx as nx
%matplotlib inline

In [2]:
from bokeh.models import Circle, ColumnDataSource, MultiLine, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8
from bokeh.transform import linear_cmap
from bokeh.io import output_notebook, show, save
output_notebook()

Loading BokehJS ...

In [3]:
from filtracion_archivos.prerrequisitos import cursos_ingenieria, diccionario_cursos_y_prerrequisitos

In [4]:
from grafos.mismo_modulo import grafo_mismo_modulo, reemplazar_siglas_con_macrosecciones
from filtracion_archivos.modulos import (cursos_y_horario,
                                         limpieza_cursos,
                                         ramos_mismo_modulo,
                                         cursos_con_macroseccion)

from grafos.drawing import dibujar_grafo

# Prerrequisitos grafo

In [5]:
# Grafo prerrequisitos
path_excel = os.path.join("excel_horarios", "Listado_materias.xlsx")
dataframe_ing = cursos_ingenieria(path_excel)
cursos = diccionario_cursos_y_prerrequisitos(dataframe_ing)

C:\Users\franc\Desktop\Universidad\5TO SEMESTRE\ICS2985_iPre\src\filtracion_archivos\prerrequisitos.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe_ing["Prerrequisitos"] = dataframe_ing["Prerrequisitos"].str.replace(
C:\Users\franc\Desktop\Universidad\5TO SEMESTRE\ICS2985_iPre\src\filtracion_archivos\prerrequisitos.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_ing["Prerrequisitos"] = dataframe_ing["Prerrequisitos"].str.replace(
C:\Users\franc\Desktop\Universidad\5TO SEMESTRE\ICS2985_iPre\src\filtracion_archivos\prerrequisitos.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [6]:
grafo_prerrequisitos = nx.from_dict_of_lists(cursos, create_using = nx.DiGraph)
grafo_prerrequisitos = grafo_prerrequisitos.reverse()

In [7]:
# Los arcos que añadiremos al grafo de los módulos. Tengo que recordar ver el tema de las macrosecciones aquí también
nuevos_arcos = dict()

for nodo in grafo_prerrequisitos.nodes:
    vecinos = [x for x in grafo_prerrequisitos.neighbors(nodo)]
    if nodo not in nuevos_arcos.keys():
        if len(vecinos) != 0:
            nuevos_arcos[nodo] = vecinos
    else:
        nuevos_arcos[nodo].extend(vecinos)
        
# Tenemos que conectar la key, con todos los nodos en la lista asociada

# Grafo modulos y macrosecciones

In [8]:
path_excel_2 = os.path.join("excel_horarios", "Listado_NRC.xlsx")
cursos_con_horario = cursos_y_horario(path_excel_2)
macrosecciones = cursos_con_macroseccion(cursos_con_horario)
grafo_modulos = grafo_mismo_modulo(path_excel_2)
grafo_modulos = reemplazar_siglas_con_macrosecciones(grafo_modulos, macrosecciones)

TypeError: reemplazar_siglas_con_macrosecciones() missing 1 required positional argument: 'cursos_a_eliminar'

In [9]:
curso_en_macrossecion = dict()

for macroseccion, sigla_secc in macrosecciones.items():
    for i in sigla_secc:
        curso_en_macrossecion[i] = macroseccion

curso_en_macrossecion

{'ICH2314-1': 'A3_MS1 - Macrosección 1',
 'ICH3314-1': 'A3_MS1 - Macrosección 1',
 'IMM2521-1': 'B0_MS1 - Macrosección 1',
 'IMM3521-1': 'B0_MS1 - Macrosección 1',
 'IIC1103-6': 'B1_MS1 - Macrosección 1',
 'IIC1103-12': 'B1_MS1 - Macrosección 1',
 'IEE2714-1': 'B2_MS1 - Macrosección 1',
 'IIC2714-1': 'B2_MS1 - Macrosección 1',
 'IIC3724-1': 'B3_MS1 - Macrosección 1',
 'IEE3724-1': 'B3_MS1 - Macrosección 1',
 'IMT3840-1': 'B6_MS1 - Macrosección 1',
 'IBM3002-1': 'B6_MS1 - Macrosección 1',
 'ICS3502-1': 'B8_MS1 - Macrosección 1',
 'ICS3313-3': 'B8_MS1 - Macrosección 1',
 'IEE3864-1': 'C1_MS1 - Macrosección 1',
 'ICS3013-3': 'C4_MS1 - Macrosección 1',
 'ICS3013-1': 'C4_MS1 - Macrosección 1',
 'ICS3013-4': 'C4_MS1 - Macrosección 1',
 'ICS3013-6': 'C4_MS1 - Macrosección 1',
 'ICS3013-5': 'C5_MS1 - Macrosección 1',
 'ICS3013-7': 'C5_MS1 - Macrosección 1',
 'ICS3013-2': 'C5_MS1 - Macrosección 1',
 'ICS2122-1': 'C8_MS1 - Macrosección 1',
 'ICS2122-2': 'C8_MS1 - Macrosección 1',
 'ICM2503-1': '

In [10]:
### Otra posible implementación. Me gusta más esta, queda más claro.
lista_nodos_modulos = list(grafo_modulos.nodes)
lista_nodos_prerrequisitos = list(grafo_prerrequisitos.nodes)
arcos_agregar = list()

contador_ramos_con_pre = 0
for i in lista_nodos_prerrequisitos:
    vecinos = [x for x in grafo_prerrequisitos.neighbors(i)]
    if len(vecinos) != 0:

        contador_ramos_con_pre += 1
        arcos_agregar.append([i, vecinos])

print(f"Hay {contador_ramos_con_pre} ramos con prerrequisitos")
contador = 0
# print(arcos_agregar)
for arco in arcos_agregar:
    ramos_con_misma_sigla = list(filter(lambda x: arco[0] in x, lista_nodos_modulos))  
    prerrequisitos_a_agregar = list()
 
    for prerreq in arco[1]:
        for sigla_seccion in lista_nodos_modulos:
            if prerreq in sigla_seccion:
                prerrequisitos_a_agregar.append(sigla_seccion)
        
    
    if len(ramos_con_misma_sigla) != 0 and len(prerrequisitos_a_agregar) != 0:
        
        # print(f"La sigla es {ramos_con_misma_sigla} y los prerrequisitos a agregar son {prerrequisitos_a_agregar}\n")
        for sigla_seccion_ramo in ramos_con_misma_sigla:
            for prerrequisito_seccion in prerrequisitos_a_agregar:
                if sigla_seccion_ramo in curso_en_macrossecion.keys():
                    
                    if prerrequisito_seccion in curso_en_macrossecion.keys():
                        grafo_modulos.add_edge(curso_en_macrossecion[sigla_seccion_ramo], curso_en_macrossecion[prerrequisito_seccion])
                    else:
                        grafo_modulos.add_edge(curso_en_macrossecion[sigla_seccion_ramo] , prerrequisito_seccion)
                
                elif prerrequisito_seccion in curso_en_macrossecion.keys() and sigla_seccion_ramo not in curso_en_macrossecion.keys():
                    grafo_modulos.add_edge(sigla_seccion_ramo, curso_en_macrossecion[prerrequisito_seccion])
                
                else:
                    grafo_modulos.add_edge(sigla_seccion_ramo, prerrequisito_seccion)
                    
        contador += 1
        
print("El valor de contador es {}", contador)

Hay 324 ramos con prerrequisitos
El valor de contador es {} 48


In [11]:
dibujar_grafo("Gramo de ramos con macrosecciones y prerrequisitos", grafo_modulos)

ValueError: failed to validate StaticLayoutProvider(id='p1083', ...).graph_layout: expected an element of Dict(Int, Seq(Any)), got {'IEE0020-1': array([-0.26700983, -0.73175523]), 'ICH0020-1': array([-0.27930742, -0.73304084]), 'ING0901-1': array([-0.24800613, -0.73492377]), 'IIC1062-1': array([-0.73022236, -0.70151967]), 'IIC2513-1': array([-0.06673282, -0.21738209]), 'ICC2204-1': array([-0.79314536,  0.21242572]), 'ICC2312-1': array([ 0.90692795, -0.43027491]), 'ICE2633-1': array([-0.60658597, -0.82810785]), 'ICE2703-1': array([ 0.09097898, -0.18378465]), 'ICH2204-1': array([0.00027098, 0.08970451]), 'ICH2304-1': array([0.10289363, 0.07059752]), 'ICM2213-1': array([-0.19105887,  0.05222423]), 'ICM2333-1': array([0.73598199, 0.55911541]), 'IMM2013-1': array([-0.36269344,  0.8721699 ]), 'IMM2043-1': array([ 0.11828691, -0.11286003]), 'IIQ2303-1': array([ 0.86398987, -0.16402328]), 'IEE2113-1': array([-0.96027142,  0.00343719]), 'ICS2123-1': array([ 0.70765069, -0.15842185]), 'ICS1113-4

In [13]:
print(len(list(nx.enumerate_all_cliques(grafo_modulos))))

9497


In [14]:
print(len(list(grafo_modulos.nodes)))

210


In [15]:
print(grafo_modulos)

Graph with 210 nodes and 623 edges


In [16]:
print(len(list(nx.find_cliques(grafo_modulos))))

96


In [17]:
G_complement = nx.complement(grafo_modulos)

In [18]:
# print(len(list(nx.find_cliques(G_complement))))

In [19]:
print(G_complement)
# Lllevaba 5 millones de cliques maximales

Graph with 210 nodes and 21322 edges


In [24]:
grafo_pygraphviz = nx.nx_agraph.to_agraph(grafo_modulos)

# Graficamos el grafo utilizando PyGraphviz
grafo_pygraphviz.layout('dot')
grafo_pygraphviz.draw('grafo.pdf')